In [1]:
import pickle
with open("data2.pickle","rb") as fr:
    data = pickle.load(fr)
data

,접수월,가맹점업종분류명_대,가맹점업종분류명,연령대코드,성별코드,광역시도명,시군구명,금액지표,건수지표,금액등급,건수등급
0,202207,보건/위생,화 장 품,70,1,강원,강릉시,145.0,89.0,2,3
1,202207,보건/위생,의료 용품,30,1,강원,강릉시,56.0,157.0,4,2
2,202207,보건/위생,의료 용품,40,1,강원,강릉시,800.0,290.0,1,1
3,202207,보건/위생,사우나,30,2,강원,강릉시,44.0,27.0,4,4
4,202207,보건/위생,사우나,60,2,강원,강릉시,45.0,39.0,4,4
...,...,...,...,...,...,...,...,...,...,...,...
3548444,202306,기타,기계 공구,50,2,충북,충주시,71.0,127.0,3,2
3548445,202306,기타,기계 공구,60,2,충북,충주시,174.0,142.0,2,2
3548446,202306,기타,비 영 리,50,1,충북,충주시,88.0,142.0,3,2
3548447,202306,기타,비 영 리,70,2,충북,충주시,51.0,40.0,4,4


In [2]:
# 파생feature생성
data['최종등급'] = data.금액등급.map(str) \
                            + data.건수등급.map(str)
data['지역'] = data.광역시도명.map(str) \
                            + data.시군구명.map(str)

In [5]:
# 최종등급 하나의 문자로 대치
data = data.replace({'최종등급' : '11'}, 'A')
data = data.replace({'최종등급' : '12'}, 'B')
data = data.replace({'최종등급' : '13'}, 'C')
data = data.replace({'최종등급' : '14'}, 'D')
data = data.replace({'최종등급' : '21'}, 'E')
data = data.replace({'최종등급' : '22'}, 'F')
data = data.replace({'최종등급' : '23'}, 'G')
data = data.replace({'최종등급' : '24'}, 'H')
data = data.replace({'최종등급' : '31'}, 'I')
data = data.replace({'최종등급' : '32'}, 'J')
data = data.replace({'최종등급' : '33'}, 'K')
data = data.replace({'최종등급' : '34'}, 'L')
data = data.replace({'최종등급' : '41'}, 'M')
data = data.replace({'최종등급' : '42'}, 'N')
data = data.replace({'최종등급' : '43'}, 'O')
data = data.replace({'최종등급' : '44'}, 'P')

In [6]:
from sklearn.preprocessing import LabelEncoder

encoder=LabelEncoder()

data['가맹점업종분류명_대']=encoder.fit_transform(data['가맹점업종분류명_대'])
data['가맹점업종분류명']=encoder.fit_transform(data['가맹점업종분류명'])
data['연령대코드']=encoder.fit_transform(data['연령대코드'])
data['성별코드']=encoder.fit_transform(data['성별코드'])
data['광역시도명']=encoder.fit_transform(data['광역시도명'])
data['시군구명']=encoder.fit_transform(data['시군구명'])

In [7]:
# feature제거
model_df = data.drop('지역', axis = 1)
model_df = model_df.drop('접수월', axis = 1)

In [9]:
model_df['최종등급']=encoder.fit_transform(model_df['최종등급'])

In [10]:
# feature제거
model_df = model_df.drop('금액등급', axis = 1)
model_df = model_df.drop('건수등급', axis = 1)

model_df = model_df.drop('금액지표', axis = 1)
model_df = model_df.drop('건수지표', axis = 1)

In [12]:
model_df #최종data

,가맹점업종분류명_대,가맹점업종분류명,연령대코드,성별코드,광역시도명,시군구명,최종등급
0,12,252,6,0,0,3,6
1,12,160,2,0,0,3,13
2,12,160,3,0,0,3,0
3,12,110,2,1,0,3,15
4,12,110,5,1,0,3,15
...,...,...,...,...,...,...,...
3548444,5,31,4,1,17,208,9
3548445,5,31,5,1,17,208,5
3548446,5,103,4,0,17,208,9
3548447,5,103,6,1,17,208,15


### model2.XGBoost

In [39]:
# data split
X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled=train_test_split(X, y,
                                         test_size=0.2, random_state=156 )


X_tr, X_val, y_tr, y_val= train_test_split(X_train_scaled, y_train_scaled,
                                         test_size=0.1, random_state=156 )

##### (1) XGBoost 성능개선 모델

In [44]:
#pip install xgboost

     --------------------------------------- 70.9/70.9 MB 16.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [46]:
from xgboost import XGBClassifier

xgb_wrapper_es = XGBClassifier(n_estimators=400, learning_rate=0.05, max_depth=3)
evals = [(X_tr, y_tr), (X_val, y_val)] #조기중단 사용시 validation 데이터를 사용한다. 모델 완성전 테스트를 위해서 train 데이터를 나눠서 검증.
xgb_wrapper_es.fit(X_tr, y_tr, early_stopping_rounds=50, eval_metric="mlogloss", 
                eval_set=evals, verbose=True)

ws50_preds = xgb_wrapper_es.predict(X_test_scaled)

[0]	validation_0-mlogloss:2.73580	validation_1-mlogloss:2.73589
[1]	validation_0-mlogloss:2.70371	validation_1-mlogloss:2.70389
[2]	validation_0-mlogloss:2.67512	validation_1-mlogloss:2.67537
[3]	validation_0-mlogloss:2.64919	validation_1-mlogloss:2.64951
[4]	validation_0-mlogloss:2.62555	validation_1-mlogloss:2.62593
[5]	validation_0-mlogloss:2.60382	validation_1-mlogloss:2.60425
[6]	validation_0-mlogloss:2.58369	validation_1-mlogloss:2.58420
[7]	validation_0-mlogloss:2.56492	validation_1-mlogloss:2.56546
[8]	validation_0-mlogloss:2.54758	validation_1-mlogloss:2.54818
[9]	validation_0-mlogloss:2.53114	validation_1-mlogloss:2.53178
[10]	validation_0-mlogloss:2.51588	validation_1-mlogloss:2.51658
[11]	validation_0-mlogloss:2.50159	validation_1-mlogloss:2.50234
[12]	validation_0-mlogloss:2.48797	validation_1-mlogloss:2.48876
[13]	validation_0-mlogloss:2.47507	validation_1-mlogloss:2.47593
[14]	validation_0-mlogloss:2.46293	validation_1-mlogloss:2.46382
[15]	validation_0-mlogloss:2.45140	

[126]	validation_0-mlogloss:2.13885	validation_1-mlogloss:2.14169
[127]	validation_0-mlogloss:2.13806	validation_1-mlogloss:2.14090
[128]	validation_0-mlogloss:2.13732	validation_1-mlogloss:2.14018
[129]	validation_0-mlogloss:2.13675	validation_1-mlogloss:2.13961
[130]	validation_0-mlogloss:2.13602	validation_1-mlogloss:2.13888
[131]	validation_0-mlogloss:2.13499	validation_1-mlogloss:2.13786
[132]	validation_0-mlogloss:2.13409	validation_1-mlogloss:2.13696
[133]	validation_0-mlogloss:2.13354	validation_1-mlogloss:2.13640
[134]	validation_0-mlogloss:2.13258	validation_1-mlogloss:2.13544
[135]	validation_0-mlogloss:2.13199	validation_1-mlogloss:2.13486
[136]	validation_0-mlogloss:2.13128	validation_1-mlogloss:2.13417
[137]	validation_0-mlogloss:2.13041	validation_1-mlogloss:2.13330
[138]	validation_0-mlogloss:2.12965	validation_1-mlogloss:2.13254
[139]	validation_0-mlogloss:2.12913	validation_1-mlogloss:2.13203
[140]	validation_0-mlogloss:2.12849	validation_1-mlogloss:2.13139
[141]	vali

[251]	validation_0-mlogloss:2.07769	validation_1-mlogloss:2.08097
[252]	validation_0-mlogloss:2.07739	validation_1-mlogloss:2.08068
[253]	validation_0-mlogloss:2.07702	validation_1-mlogloss:2.08031
[254]	validation_0-mlogloss:2.07671	validation_1-mlogloss:2.08000
[255]	validation_0-mlogloss:2.07648	validation_1-mlogloss:2.07977
[256]	validation_0-mlogloss:2.07628	validation_1-mlogloss:2.07957
[257]	validation_0-mlogloss:2.07595	validation_1-mlogloss:2.07924
[258]	validation_0-mlogloss:2.07573	validation_1-mlogloss:2.07902
[259]	validation_0-mlogloss:2.07534	validation_1-mlogloss:2.07864
[260]	validation_0-mlogloss:2.07491	validation_1-mlogloss:2.07821
[261]	validation_0-mlogloss:2.07459	validation_1-mlogloss:2.07790
[262]	validation_0-mlogloss:2.07434	validation_1-mlogloss:2.07765
[263]	validation_0-mlogloss:2.07413	validation_1-mlogloss:2.07745
[264]	validation_0-mlogloss:2.07364	validation_1-mlogloss:2.07695
[265]	validation_0-mlogloss:2.07333	validation_1-mlogloss:2.07664
[266]	vali

[376]	validation_0-mlogloss:2.04332	validation_1-mlogloss:2.04698
[377]	validation_0-mlogloss:2.04303	validation_1-mlogloss:2.04668
[378]	validation_0-mlogloss:2.04278	validation_1-mlogloss:2.04644
[379]	validation_0-mlogloss:2.04260	validation_1-mlogloss:2.04625
[380]	validation_0-mlogloss:2.04232	validation_1-mlogloss:2.04598
[381]	validation_0-mlogloss:2.04216	validation_1-mlogloss:2.04583
[382]	validation_0-mlogloss:2.04190	validation_1-mlogloss:2.04558
[383]	validation_0-mlogloss:2.04170	validation_1-mlogloss:2.04537
[384]	validation_0-mlogloss:2.04149	validation_1-mlogloss:2.04516
[385]	validation_0-mlogloss:2.04124	validation_1-mlogloss:2.04492
[386]	validation_0-mlogloss:2.04099	validation_1-mlogloss:2.04467
[387]	validation_0-mlogloss:2.04082	validation_1-mlogloss:2.04451
[388]	validation_0-mlogloss:2.04065	validation_1-mlogloss:2.04433
[389]	validation_0-mlogloss:2.04029	validation_1-mlogloss:2.04397
[390]	validation_0-mlogloss:2.04005	validation_1-mlogloss:2.04374
[391]	vali

In [1]:
# 하이퍼파라미터 튜닝(성능개선)
#pip install hyperopt

In [62]:
from hyperopt import hp

#하이퍼파라미터 목록
xgb_search_space = {'max_depth': hp.quniform('max_depth', 5, 15, 1), 
                    'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
                    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 0.95),
                    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2)
}

In [63]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import numpy as np
from xgboost import XGBClassifier

# 목적 함수 설정.
# 추후 fmin()에서 입력된 search_space값으로 XGBClassifier 교차 검증 학습 후 -1* macro avg roc_auc 평균 값을 반환.
def objective_func(search_space):
    xgb_clf = XGBClassifier(n_estimators=100, max_depth=int(search_space['max_depth']),
                            min_child_weight=int(search_space['min_child_weight']),
                            colsample_bytree=search_space['colsample_bytree'],
                            learning_rate=search_space['learning_rate'],
                            objective='multi:softmax',  # 다중 클래스 분류를 위한 설정
                            num_class=16  # 클래스 개수 설정
                            )
    
    # 3개 k-fold 방식으로 평가된 macro avg roc_auc 지표를 담는 list
    roc_auc_list= []
    
    # 3개 k-fold방식 적용
    kf = KFold(n_splits=3)
    for tr_index, val_index in kf.split(X_train_scaled):
        X_tr, y_tr = X_train_scaled.iloc[tr_index], y_train_scaled.iloc[tr_index]
        X_val, y_val = X_train_scaled.iloc[val_index], y_train_scaled.iloc[val_index]
        
        xgb_clf.fit(X_tr, y_tr, early_stopping_rounds=30, eval_metric='mlogloss',  # 다중 클래스 분류를 위한 설정
                    eval_set=[(X_tr, y_tr), (X_val, y_val)])
        
        # predict_proba에서 각 클래스에 대한 예측 확률 추출
        y_pred_proba = xgb_clf.predict_proba(X_val)
        
        # 각 클래스에 대한 ROC AUC 계산 후 macro avg 계산
        macro_avg_roc_auc = roc_auc_score(y_val, y_pred_proba, average='macro', multi_class = 'ovr')
        roc_auc_list.append(macro_avg_roc_auc)
    
    # 3개 k-fold로 계산된 macro avg roc_auc값의 평균값을 반환하되, 
    # HyperOpt는 목적함수의 최소값을 위한 입력값을 찾으므로 -1을 곱한 뒤 반환.
    return -1 * np.mean(roc_auc_list)

In [64]:
from hyperopt import fmin, tpe, Trials

trials = Trials()

best = fmin(fn=objective_func,
            space=xgb_search_space,
            algo=tpe.suggest,
            max_evals=50,  # 최대 반복 횟수지정
            trials=trials, rstate=np.random.default_rng(seed=30)
            )

print('best:', best)

  0%|                                                                           | 0/50 [00:00<?, ?trial/s, best loss=?]

C:\Users\yunse\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\yunse\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



[0]	validation_0-mlogloss:2.70162	validation_1-mlogloss:2.70157                                                        
[1]	validation_0-mlogloss:2.63915	validation_1-mlogloss:2.63904                                                        
[2]	validation_0-mlogloss:2.58998	validation_1-mlogloss:2.58975                                                        
[3]	validation_0-mlogloss:2.54913	validation_1-mlogloss:2.54890                                                        
[4]	validation_0-mlogloss:2.51189	validation_1-mlogloss:2.51161                                                        
[5]	validation_0-mlogloss:2.47378	validation_1-mlogloss:2.47345                                                        
[6]	validation_0-mlogloss:2.44224	validation_1-mlogloss:2.44193                                                        
[7]	validation_0-mlogloss:2.41394	validation_1-mlogloss:2.41360                                                        
[8]	validation_0-mlogloss:2.38650	valida

C:\Users\yunse\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\yunse\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



[0]	validation_0-mlogloss:2.70142	validation_1-mlogloss:2.70165                                                        
[1]	validation_0-mlogloss:2.63955	validation_1-mlogloss:2.64000                                                        
[2]	validation_0-mlogloss:2.59014	validation_1-mlogloss:2.59077                                                        
[3]	validation_0-mlogloss:2.54921	validation_1-mlogloss:2.54999                                                        
[4]	validation_0-mlogloss:2.51236	validation_1-mlogloss:2.51329                                                        
[5]	validation_0-mlogloss:2.47402	validation_1-mlogloss:2.47512                                                        
[6]	validation_0-mlogloss:2.44240	validation_1-mlogloss:2.44365                                                        
[7]	validation_0-mlogloss:2.41426	validation_1-mlogloss:2.41562                                                        
[8]	validation_0-mlogloss:2.38670	valida

...

[0]	validation_0-mlogloss:2.57571	validation_1-mlogloss:2.57690                                                        
[1]	validation_0-mlogloss:2.45842	validation_1-mlogloss:2.46079                                                        
[2]	validation_0-mlogloss:2.37514	validation_1-mlogloss:2.37850                                                        
[3]	validation_0-mlogloss:2.30222	validation_1-mlogloss:2.30639                                                        
[4]	validation_0-mlogloss:2.24779	validation_1-mlogloss:2.25289                                                        
[5]	validation_0-mlogloss:2.19886	validation_1-mlogloss:2.20474                                                        
[6]	validation_0-mlogloss:2.15886	validation_1-mlogloss:2.16560                                                        
[7]	validation_0-mlogloss:2.12354	validation_1-mlogloss:2.13102                                                        
[8]	validation_0-mlogloss:2.09290	valida

C:\Users\yunse\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\yunse\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



[0]	validation_0-mlogloss:2.57786	validation_1-mlogloss:2.57959                                                        
[1]	validation_0-mlogloss:2.45869	validation_1-mlogloss:2.46176                                                        
[2]	validation_0-mlogloss:2.37608	validation_1-mlogloss:2.38052                                                        
[3]	validation_0-mlogloss:2.30291	validation_1-mlogloss:2.30850                                                        
[4]	validation_0-mlogloss:2.24793	validation_1-mlogloss:2.25456                                                        
[5]	validation_0-mlogloss:2.19850	validation_1-mlogloss:2.20620                                                        
[6]	validation_0-mlogloss:2.15829	validation_1-mlogloss:2.16698                                                        
[7]	validation_0-mlogloss:2.12354	validation_1-mlogloss:2.13309                                                        
[8]	validation_0-mlogloss:2.09316	valida

C:\Users\yunse\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\yunse\anaconda3\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



[0]	validation_0-mlogloss:2.57775	validation_1-mlogloss:2.57932                                                        
[1]	validation_0-mlogloss:2.45848	validation_1-mlogloss:2.46121                                                        
[2]	validation_0-mlogloss:2.37449	validation_1-mlogloss:2.37829                                                        
[3]	validation_0-mlogloss:2.30259	validation_1-mlogloss:2.30737                                                        
[4]	validation_0-mlogloss:2.24813	validation_1-mlogloss:2.25390                                                        
[5]	validation_0-mlogloss:2.19964	validation_1-mlogloss:2.20621                                                        
[6]	validation_0-mlogloss:2.15950	validation_1-mlogloss:2.16694                                                        
[7]	validation_0-mlogloss:2.12464	validation_1-mlogloss:2.13281                                                        
[8]	validation_0-mlogloss:2.09442	valida

In [66]:
# 최적의 파라미터로 학습
xgb_clf = XGBClassifier(n_estimators=500, learning_rate=round(best['learning_rate'], 5),
                        max_depth=int(best['max_depth']), min_child_weight=int(best['min_child_weight']), 
                        colsample_bytree=round(best['colsample_bytree'], 5),
                        objective='multi:softmax',  # 다중 클래스 분류를 위한 설정
                        num_class=16  # 클래스 개수 설정
                       )

xgb_clf.fit(X_tr, y_tr, early_stopping_rounds=100, 
            eval_metric="mlogloss",  # 다중 클래스 분류를 위한 설정
            eval_set=[(X_tr, y_tr), (X_val, y_val)])

# 각 클래스에 대한 예측 확률 추출
y_pred_proba = xgb_clf.predict_proba(X_test_scaled)

# 각 클래스에 대한 ROC AUC 계산 후 micro avg 계산
macro_avg_roc_auc = roc_auc_score(y_test_scaled, y_pred_proba, average='macro', multi_class = 'ovr')
print('macro Avg ROC AUC: {0:.4f}'.format(macro_avg_roc_auc))

[0]	validation_0-mlogloss:2.52251	validation_1-mlogloss:2.53071
[1]	validation_0-mlogloss:2.37199	validation_1-mlogloss:2.38596
[2]	validation_0-mlogloss:2.26469	validation_1-mlogloss:2.28432
[3]	validation_0-mlogloss:2.17480	validation_1-mlogloss:2.19919
[4]	validation_0-mlogloss:2.10562	validation_1-mlogloss:2.13426
[5]	validation_0-mlogloss:2.04530	validation_1-mlogloss:2.07763
[6]	validation_0-mlogloss:1.99358	validation_1-mlogloss:2.02978
[7]	validation_0-mlogloss:1.94662	validation_1-mlogloss:1.98620
[8]	validation_0-mlogloss:1.90443	validation_1-mlogloss:1.94750
[9]	validation_0-mlogloss:1.86951	validation_1-mlogloss:1.91547
[10]	validation_0-mlogloss:1.83821	validation_1-mlogloss:1.88713
[11]	validation_0-mlogloss:1.80582	validation_1-mlogloss:1.85834
[12]	validation_0-mlogloss:1.77842	validation_1-mlogloss:1.83395
[13]	validation_0-mlogloss:1.75214	validation_1-mlogloss:1.81090
[14]	validation_0-mlogloss:1.73102	validation_1-mlogloss:1.79252
[15]	validation_0-mlogloss:1.71062	

[126]	validation_0-mlogloss:1.18946	validation_1-mlogloss:1.39710
[127]	validation_0-mlogloss:1.18774	validation_1-mlogloss:1.39614
[128]	validation_0-mlogloss:1.18623	validation_1-mlogloss:1.39525
[129]	validation_0-mlogloss:1.18446	validation_1-mlogloss:1.39424
[130]	validation_0-mlogloss:1.18225	validation_1-mlogloss:1.39296
[131]	validation_0-mlogloss:1.18064	validation_1-mlogloss:1.39215
[132]	validation_0-mlogloss:1.17902	validation_1-mlogloss:1.39123
[133]	validation_0-mlogloss:1.17725	validation_1-mlogloss:1.39018
[134]	validation_0-mlogloss:1.17529	validation_1-mlogloss:1.38896
[135]	validation_0-mlogloss:1.17382	validation_1-mlogloss:1.38809
[136]	validation_0-mlogloss:1.17211	validation_1-mlogloss:1.38716
[137]	validation_0-mlogloss:1.17047	validation_1-mlogloss:1.38631
[138]	validation_0-mlogloss:1.16876	validation_1-mlogloss:1.38534
[139]	validation_0-mlogloss:1.16716	validation_1-mlogloss:1.38442
[140]	validation_0-mlogloss:1.16539	validation_1-mlogloss:1.38331
[141]	vali

[251]	validation_0-mlogloss:1.04483	validation_1-mlogloss:1.32316
[252]	validation_0-mlogloss:1.04388	validation_1-mlogloss:1.32270
[253]	validation_0-mlogloss:1.04317	validation_1-mlogloss:1.32242
[254]	validation_0-mlogloss:1.04224	validation_1-mlogloss:1.32194
[255]	validation_0-mlogloss:1.04158	validation_1-mlogloss:1.32167
[256]	validation_0-mlogloss:1.04081	validation_1-mlogloss:1.32134
[257]	validation_0-mlogloss:1.03996	validation_1-mlogloss:1.32100
[258]	validation_0-mlogloss:1.03929	validation_1-mlogloss:1.32078
[259]	validation_0-mlogloss:1.03860	validation_1-mlogloss:1.32055
[260]	validation_0-mlogloss:1.03799	validation_1-mlogloss:1.32033
[261]	validation_0-mlogloss:1.03735	validation_1-mlogloss:1.32010
[262]	validation_0-mlogloss:1.03658	validation_1-mlogloss:1.31975
[263]	validation_0-mlogloss:1.03593	validation_1-mlogloss:1.31949
[264]	validation_0-mlogloss:1.03518	validation_1-mlogloss:1.31921
[265]	validation_0-mlogloss:1.03426	validation_1-mlogloss:1.31880
[266]	vali

[376]	validation_0-mlogloss:0.97181	validation_1-mlogloss:1.29758
[377]	validation_0-mlogloss:0.97143	validation_1-mlogloss:1.29756
[378]	validation_0-mlogloss:0.97104	validation_1-mlogloss:1.29748
[379]	validation_0-mlogloss:0.97073	validation_1-mlogloss:1.29746
[380]	validation_0-mlogloss:0.97022	validation_1-mlogloss:1.29730
[381]	validation_0-mlogloss:0.96967	validation_1-mlogloss:1.29711
[382]	validation_0-mlogloss:0.96914	validation_1-mlogloss:1.29700
[383]	validation_0-mlogloss:0.96871	validation_1-mlogloss:1.29688
[384]	validation_0-mlogloss:0.96828	validation_1-mlogloss:1.29677
[385]	validation_0-mlogloss:0.96797	validation_1-mlogloss:1.29673
[386]	validation_0-mlogloss:0.96759	validation_1-mlogloss:1.29668
[387]	validation_0-mlogloss:0.96732	validation_1-mlogloss:1.29667
[388]	validation_0-mlogloss:0.96683	validation_1-mlogloss:1.29652
[389]	validation_0-mlogloss:0.96635	validation_1-mlogloss:1.29635
[390]	validation_0-mlogloss:0.96591	validation_1-mlogloss:1.29628
[391]	vali

ValueError: average must be one of ('macro', 'weighted') for multiclass problems

In [78]:
y_pred = xgb_clf.predict(X_test_scaled)

In [67]:
macro_avg_roc_auc = roc_auc_score(y_test_scaled, y_pred_proba, average='macro', multi_class = 'ovr')
print('macro Avg ROC AUC: {0:.4f}'.format(macro_avg_roc_auc))

macro Avg ROC AUC: 0.9097


In [79]:
accuracy = accuracy_score(y_test_scaled, y_pred)
print('Accuracy: {0:.4f}'.format(accuracy))

Accuracy: 0.5826


In [68]:
# 피처 중요도 출력
feature_importances = xgb_clf.feature_importances_
sorted_indices = np.argsort(feature_importances)[::-1]  # 중요도가 높은 순서대로 정렬

print("Feature Importances:")
for i, index in enumerate(sorted_indices):
    print("{0}: {1:.4f}".format(X_test_scaled.columns[index], feature_importances[index]))

Feature Importances:
가맹점업종분류명: 0.2677
연령대코드: 0.1791
가맹점업종분류명_대: 0.1789
시군구명: 0.1709
광역시도명: 0.1236
성별코드: 0.0798


In [ ]:
y_pred_proba = xgb_clf.predict_proba(X_test_scaled)

# 각 클래스에 대한 ROC AUC 계산 후 micro avg 계산
macro_avg_roc_auc = roc_auc_score(y_test_scaled, y_pred_proba, average='macro', multi_class = 'ovr')
print('macro Avg ROC AUC: {0:.4f}'.format(macro_avg_roc_auc))

In [76]:
# 새로운 입력 데이터 준비 (특성 값은 실제 데이터에 맞게 수정)
input_data = np.array([12, 110, 2, 1, 0, 3])  # 각 샘플의 특성을 나타내는 벡터

input_data_reshaped = input_data.reshape(1, -1)

# 예측 결과 얻기
predicted_probs = xgb_clf.predict_proba(input_data_reshaped)  # 클래스별 예측 확률을 반환

# 가장 확률이 높은 클래스 선택
predicted_class = np.argmax(predicted_probs)  # 가장 높은 확률을 가진 클래스 선택

print(predicted_class)

15
